In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Bidirectional
from modules.utils import  change_X_data_2


In [2]:
X = pd.read_csv("/Users/elouan/Data/Data_ENSxCFM/X_train.csv")
y = pd.read_csv("/Users/elouan/Data/Data_ENSxCFM/y_train.csv")

In [3]:
X2 = change_X_data_2(X)


/Users/elouan/anaconda3/envs/challenge_ENS_env_CFM/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [4]:
X3 = np.array([np.array(row) for row in X2["column 1"]])

In [5]:
y1 = y.drop(columns=["obs_id"])

In [6]:
y2 = np.array(y1["eqt_code_cat"])

In [10]:
print(np.any(np.isnan(X3)))  # Check features for NaNs
print(np.any(np.isnan(y2)))

False
False


In [7]:
X3 = np.nan_to_num(X3)

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((X3, y2))

In [9]:
# Calculate sizes for train and validation sets
dataset_size = len(X3)  # Total number of samples in your dataset
train_size = int(0.8 * dataset_size)

# Shuffle the dataset if needed
dataset = dataset.shuffle(buffer_size=dataset_size)

# Split the dataset
train_dataset = dataset.take(train_size)       # Take the first 80%
val_dataset = dataset.skip(train_size)         # Skip the first 80%, take the remaining 20%

# Batch both datasets if needed
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [25]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [   Bidirectional(GRU(128, return_sequences=True, input_shape = (100, 11) )),            
        Dropout(0.2),
        Bidirectional(GRU(64)),
        Dropout(0.2),
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(32, activation = 'relu',   name = "L2"),
        Dense(24, activation = 'softmax',   name = "output"),

    ], name = "second_multiclass_model" 
)

/Users/elouan/anaconda3/envs/challenge_ENS_env_CFM/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
model = tf.keras.models.load_model("../models/second_model_4(0.7930).h5")

In [16]:
model.summary()

Model: "second_multiclass_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_8 (Bidirectional) │ (None, 100, 256)       │       108,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ (None, 128)            │       123,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L1 (Dense)                      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L2 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 24)             │           792 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 729,194 (2.78 MB)

 Trainable params: 243,064 (949.47 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 486,130 (1.85 MB)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000015625),
    metrics=['accuracy']
)




reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Choose the metric to monitor
    factor=0.5,          # Factor by which to reduce the learning rate
    patience=3,          # Number of epochs with no improvement before reducing
    min_lr=1e-6          # Minimum learning rate
)

checkpoint = ModelCheckpoint(
    '../models/Final_training/Second_model_5.keras',             # File to save the best model
    monitor='val_accuracy',       # Metric to monitor for improvement
    mode='max',                   # Mode 'max' for accuracy (since higher is better)
    save_best_only=True,          # Save only when there is an improvement
    verbose=1,                    # Print message when saving
                    
)


history = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs=10,
    callbacks=[reduce_lr, checkpoint]
)

Epoch 1/10
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - accuracy: 0.7981 - loss: 0.5677
Epoch 1: val_accuracy improved from -inf to 0.85239, saving model to ../models/Final_training/Second_model_5.keras
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 973s 241ms/step - accuracy: 0.7981 - loss: 0.5677 - val_accuracy: 0.8524 - val_loss: 0.4263 - learning_rate: 1.5625e-05
Epoch 2/10
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.7925 - loss: 0.5774
Epoch 2: val_accuracy did not improve from 0.85239
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 978s 243ms/step - accuracy: 0.7925 - loss: 0.5774 - val_accuracy: 0.8515 - val_loss: 0.4286 - learning_rate: 1.5625e-05
Epoch 3/10
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.7997 - loss: 0.5655
Epoch 3: val_accuracy improved from 0.85239 to 0.85283, saving model to ../models/Final_training/Second_model_5.keras
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 993s 247ms/step - accuracy: 0.7997 - loss: 0.5655 - val_accuracy: 0.8528 - val_loss: 0.4228 - learning_rate: 1.56

In [18]:
model.save("../models/second_model_6(0.85407).h5") 